In [1]:
import torch
import torch.nn as nn 
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [2]:
# 定义一些参数
input_size=5
output_size=2

batch_size=30
data_size=100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# 制作一个虚拟(随机）数据集
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

In [4]:
# 设立一个简单模型
# 作为演示，我们的模型只接受一个输入，执行一个线性操作，然后得到结果
# 可以在任何模型上使用DataParallel
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

In [6]:
# 创建模型
# 检测是否有多个GPU,如有，则使用nn.DataParallel来包装我们的模型，然后放至GPU
model = Model(input_size, output_size)
if torch.cuda.device_count() > 0: 
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)

Let's use 1 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [7]:
# 1个GPU的运行结果
for data in rand_loader: 
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


In [ ]:
# DataParallel自动的划分数据，并将作业顺序发送到多个GPU上的多个模型
# DataParallel会在每个模型完成作业后，收集与合并结果然后返回给你